# US presidential statistics
## Simon Repko, Lukas Petrasek
### IES FSS CU
### 31.5.2019

This notebook serves as a demonstration of a school project whose goal is to achieve the following:
* scrape web pages to get historical data on US presidents
* manipulate the data into a form suitable for being visualized
* make vizualizations based the data

In [ ]:
# TODO: import packages

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
# TODO: initialize the scraping class and apply the methods necessary to get the data here

In [ ]:
response = requests.get('https://millercenter.org/')
soup = BeautifulSoup(response.text,'html.parser')
name_list_1 = soup.find('nav',{'aria-labelledby':'block-mainnavigation-3-menu'})
name_list_2 = name_list_1.find_all('ul',{'class':'submenu'})[1]
name_list_2

pres_dict = {}
for pres in name_list_2.find_all('a'):
    pres_dict[pres.text] = pres['href']

len(pres_dict)    

pres_dict

In [ ]:
#NEW attr that will create list and remove non-needed items
def list_clean(l): # l is object with type 'bs4.element.Tag'
    a = list(l.children)
    a = [x for x in a if x != '\n']
    a.pop(0)
    return a

In [ ]:
data_presidents = {}
i=1 

for name,href in pres_dict.items():
    html = "".join(['https://millercenter.org',href])
    response = requests.get(html)
    soup = BeautifulSoup(response.text,'html.parser')

    #1 extraction of FAST FACTS dashboard of individual presidents
    details = soup.find('div',{'class':'president-main-wrapper'}).find('div',{'class':'fast-facts-wrapper'})   
 
    pres_details = {}
    for det in list_clean(details):     
        pres_details[det.label.text] = det.div.text
        #print(det,det.div.text) 
        #print('*')
        
    #2 brief description of the president
    pres_desc = {}
    pres_desc['Description'] = soup.find('div',{'class':'copy-wrapper'}).p.text   
    
    #3 famous quote of the president
    quote = {}
    quote['Quote'] = soup.find('blockquote',{'class':'president-quote'}).contents[0]
    
    #4 number of KEY EVENTS that happened during office
    key_events = soup.find('div',{'class':'sub-nav-region'}).find_all('a')[1]

    html_1 = "".join(['https://millercenter.org',key_events['href']]) # getting into the list of key events of president 
    response_1 = requests.get(html_1)
    soup_1 = BeautifulSoup(response_1.text,'html.parser')
    ke_count = {}
    #count of number of major events that happened during being in office - key events_count_X    
    try: # D. Trump page has no information about major events hence we included error exception     
        ke_c_1 = len(soup_1.find('div',{'class':'article-wysiwyg-body'}).find_all('strong')) 
        ke_c_2 = len(soup_1.find('div',{'class':'article-wysiwyg-body'}).find_all('b'))
        ke_count['Number of major events'] = ke_c_1 + ke_c_2
    except AttributeError:
        ke_count['Number of major events'] = 0
        pass
    
    data_presidents[name] = {**pres_details,**pres_desc,**quote,**ke_count} #merge of the dictionaries
    print(i)
    i=i+1
    
data_presidents

In [ ]:
# Final table with data extracted from https://millercenter.org
pd.DataFrame(data_presidents)

In [ ]:
# TODO: manipulate the data here

In [ ]:
# TODO: make the visualizations here

In [ ]:
# TODO: conclude here?